## Data Loading 

In [1]:
import numpy as np 
import ipywidgets as widgets
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import sklearn.linear_model
from sklearn.preprocessing import PolynomialFeatures

In [2]:
fp = os.path.join('data/Hurricane_Irene_and_the_Hudson_River.xlsx')
do_data = pd.read_excel(fp, sheet_name = 5).drop(['Piermont D.O. (ppm)'], axis = 1)
rainfall_data = pd.read_excel(fp, sheet_name = 'Rainfall').drop(['Piermont  Rainfall Daily Accumulation (Inches)'], axis = 1)
turbidity_data = pd.read_excel(fp, sheet_name = 'Turbidity').drop(['Piermont Turbidity in NTU'], axis = 1)

## Data Cleaning

Since we read in our data through multiple excel sheets, we need to merge our different dtaaframes. The different dat sheets all have a column for date, so we can merge our data on this. To better call on the dqata in our analysis, we camn change the column names. Finally, we will update the date to be of the datetime type, and set rthe date as the index.

In [3]:
# Merge the two datasets
data = do_data.merge(rainfall_data, on = 'Date Time (ET)')
data = data.merge(turbidity_data, on = "Date Time (ET)")
data.head(3)


# Update the column names
data.columns = ['date', 'albany_do', 'norrie_do', 'albany_rainfall', 'norrie_rainfall', 'albany_turbidity', 'norrie_turbidity']

# Convert data to datetime format and set it as index
data['date'] = pd.to_datetime(data['date'])

# Set date as the index
data.set_index('date', inplace = True)

data.head()

,albany_do,norrie_do,albany_rainfall,norrie_rainfall,albany_turbidity,norrie_turbidity
date,,,,,,
2011-08-25 00:00:00,7.68,7.81,0.0,0.0,4.0,9.3
2011-08-25 00:15:00,7.60,7.73,0.0,0.0,3.9,8.4
2011-08-25 00:30:00,7.57,7.63,0.0,0.0,4.3,7.9
2011-08-25 00:45:00,7.72,7.67,0.0,0.0,4.7,8.1
2011-08-25 01:00:00,7.74,7.63,0.0,0.0,4.4,8.4


## Multiple Linear Regression

In [4]:
# Define predictors and the target variable
x = data[['albany_do', 'albany_rainfall']]
y = data[['albany_turbidity']]

# Split the data into training and testing sets
x_train, x_test, y_train, y_test, = train_test_split(x, y, test_size = 0.3, random_state = 42)

# Create and fit the model
model = LinearRegression()
model.fit(x_train, y_train)

# Predict and evaluate
y_pred = model.predict(x_test)


rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"R-squared: {r2_score(y_test, y_pred)}")


RMSE: 221.9143474905527
R-squared: 0.4907389518457509


## Create a widget to visualize different models

In [5]:
# Create a widget for selecting predictors
predictor_selector = widgets.SelectMultiple(
    options = data.columns, 
    value = (data.columns[0], ),
    description = 'Predictors'
)

# Create a dropdown for selecting the target variable
target_selector = widgets.Dropdown(
    options = data.columns,
    value = data.columns[1],
    description = 'Target'
)

# Button to evaluate the model 
evaluate_button = widgets.Button(description = 'Evaluate Model')

# Output widget to display results
output = widgets.Output()

# Define the function to handle button clicks
def evaluate_model(b):
    with output:
        clear_output(wait = True) # Clear output of display area
        
        # Make sure the target is not in the predictors
        selected_predictors = [item for item in predictor_selector.value]
        if target_selector.value in selected_predictors :
            proint ("Target variable must not be in selcted predictors.")
            return
        
        # Prepare the data 
        x = data[[selected_predictors]]
        y = data[[target_selector.value]]
        
        # Split data into training and testing sets
        x_train, x_test, y_train, y_test, = train_test_split(x, y, test_size = 0.3, random_state = 42)
        
        
        # Create and fit the model
        model = LinearRegression()
        model.fit(x_train, y_train)
        
        # Predict and calculate R^2 and MSE
        y_pred = model.predict(x_test)
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        
        
        # Display the R^2 score and MSE
        print(f"R^2: {r2:.4f}")
        print(f"MSE: {mse:.4f}")
        
# Display the widgets and connect the button to the function
display(predictor_selector, target_selector, evaluate_button, output)
evaluate_button.on_click(evaluate_model)

SelectMultiple(description='Predictors', index=(0,), options=('albany_do', 'norrie_do', 'albany_rainfall', 'no…

Dropdown(description='Target', index=1, options=('albany_do', 'norrie_do', 'albany_rainfall', 'norrie_rainfall…

Button(description='Evaluate Model', style=ButtonStyle())

Output()